Bitcoin Efficiency Calculations
=================

In [3]:
import requests
import csv
import statistics
import time

res = requests.get("https://api.blockchain.info/charts/hash-rate?cors=true&format=csv&lang=de").text
lines = list(csv.reader(res.splitlines()))

# Average over the last 60 days and convert from THash/s to Hash/s
hashrate_60d_avg = statistics.mean(float(rate_s)*1e12 for _, rate_s in lines[-60:]) # Hash/s

In [4]:

# Very optimistic lower bound, ignoring older hardware, cooling etc. pp.
power_consumption = 1375/14e12 # W/(Hash/s) → J/Hash

# 1J = 1Ws → Hash/s * J/Hash = J/s = Ws/s = W
network_power_consumption = hashrate_60d_avg * power_consumption # W

print('Total network power consumption (estimated lower bound):', network_power_consumption*1e-6, 'MW')

Total network power consumption (estimated lower bound): 177.46252592029907 MW


* https://en.bitcoin.it/wiki/Mining_hardware_comparison#cite_note-AntMinerS9-9
* https://www.bitmaintech.com/productDetail.htm?pid=0002016052907243375530DcJIoK0654
* https://blockchain.info/api/blockchain_api

In [5]:
# Fetch blockchain.info data on the last [n] blocks
import sqlite3

db = sqlite3.connect('blockinfo.db')
db.execute('CREATE TABLE IF NOT EXISTS blockinfo (idx INTEGER, info TEXT)')

def fetch_blocks(n=100): # approx. 1k blocks in 7d
    latest = requests.get('https://blockchain.info/latestblock').json()['block_index']
    with db as conn:
        for idx in range(latest, latest-n, -1):
            if conn.execute('SELECT count(1) FROM blockinfo WHERE idx=?', (idx,)).fetchone()[0] == 0:
                print('fetching:', idx, flush=True)
                time.sleep(1)
                info = requests.get('https://blockchain.info/block-index/{}?format=json'.format(idx)).text
                conn.execute('INSERT INTO blockinfo VALUES (?, ?)', (idx, info))
#            else:
#                print('skipping:', idx, flush=True)
    print('done.')

def blocks(lower, upper, conn=db):
    for idx, in conn.execute('SELECT idx FROM blockinfo WHERE idx BETWEEN ? AND ? ORDER BY idx', (lower, upper)):
        yield json.loads(conn.execute('SELECT info FROM blockinfo WHERE idx=?', (idx,)).fetchone()[0])

In [6]:
#fetch_blocks(1000)

In [7]:
import json
import statistics
def avg_tx_last_n(n=1000):
    with db as conn:
        last_idx = db.execute('SELECT MAX(idx) FROM blockinfo').fetchone()[0]
        data = [
            (len(b['tx']), b['time']) for b in blocks(last_idx-n, last_idx, conn)
        ]
        avg_tx_per_block = statistics.mean(ntx for ntx, time in data)
        avg_seconds_per_block = (data[-1][1] - data[0][1])/n
    avg_tx_per_second = avg_tx_per_block / avg_seconds_per_block
    return avg_tx_per_second

avg_tx_last_1000 = avg_tx_last_n(n=1000)
avg_tx_last_1000

2.823744356770102

In [8]:
energy_per_tx = network_power_consumption * 1 / avg_tx_last_1000 # J
kwh_per_tx = energy_per_tx / 1000 / 3600 # Ws → Wh, Wh → kWh
print('Total network energy consumption per transaction (estimated lower bound): {:.02f} MJ = {:.02f} kWh'.format(
        energy_per_tx*1e-6, kwh_per_tx))

Total network energy consumption per transaction (estimated lower bound): 62.85 MJ = 17.46 kWh


In [9]:
kcal_per_day_adult_male = 2600
J_per_day_adult_male = 180/43 * 1000 * kcal_per_day_adult_male
adult_males_nourished_1d_per_tx = energy_per_tx/J_per_day_adult_male
print('Days an adult male could be nourished from the energy a single transaction consumes (approximated, dependent on person):', adult_males_nourished_1d_per_tx)

Days an adult male could be nourished from the energy a single transaction consumes (approximated, dependent on person): 5.774360385497753


* https://en.wikipedia.org/wiki/Calorie
* https://en.wikipedia.org/wiki/Food_energy 

In [10]:
bn_info = requests.get('https://bitnodes.21.co/api/v1/snapshots/latest/').json()

In [11]:
import collections
countries = collections.Counter()
total_nodes = bn_info['total_nodes']
for pver, uagent, conntime, svcs, height, host, city, country, lat, lon, tz, asn, org in bn_info['nodes'].values():
    countries[country] += 1
country_fractions = [ (country, count/total_nodes) for country, count in countries.most_common() ]
cf_idx = 10
print(country_fractions[:cf_idx], sum(frac for country, frac in country_fractions[:cf_idx]))

[('US', 0.27481196110805356), ('DE', 0.17262887543569988), ('FR', 0.08145294441386902), ('NL', 0.05723720418271877), ('CA', 0.043845166024582646), ('GB', 0.043478260869565216), (None, 0.033571821684094665), ('CN', 0.03265455879655109), ('RU', 0.028985507246376812), ('CH', 0.016877637130801686)] 0.7855439368923133


* https://bitnodes.21.co/api/

In [12]:
import csv

iso_3166_data = requests.get('http://www.iso.org/iso/home/standards/country_codes/country_names_and_code_elements_txt-temp.htm')
iso_3166_data = iso_3166_data.text.splitlines()[1:-1]
iso_3166_1_alpha_2_to_name = {
    cc.lower(): name.lower() for name, cc in ( line.split(';') for line in iso_3166_data )
}

* https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2

In [13]:
from bs4 import BeautifulSoup

#energy_by_coutry_xls = requests.get('http://www.iea.org/media/statistics/surveys/electricity/mes.xls').content
energy_by_coutry_table = requests.get('http://wdi.worldbank.org/AjaxDownload/FileDownloadHandler.ashx?filename=3.7_Electricity_production_sources_and_access.xls&filetype=excel').content
# don't ask why, but the above link actually returns readable html. And who am I to complain that I get HTML to parse instead of an Excel file? =)
soup = BeautifulSoup(energy_by_coutry_table, 'lxml')
tbody = soup.find('tbody') # data table body
energy_by_country = {
    cname.text.lower(): [ float(col.text.replace(',', ''))/100 if col.text != '..' else None for col in numbers ]
            for cname, *numbers in ( row.find_all('td') for row in tbody.find_all('tr') )
}

* http://wdi.worldbank.org/table/3.7#

In [14]:
import numpy as np
from IPython.display import display, HTML

fracs = np.zeros(6, dtype=np.float) # coal, gas, oil, hydro, renewable, nuclear
accounted_total = 0
for cc, frac_total in country_fractions:
    if cc is None:
        break
    twh_total, *cfracs, access = energy_by_country[iso_3166_1_alpha_2_to_name[cc.lower()]]
    accounted_total += frac_total
    fracs += np.array(cfracs)*frac_total
fracs /= accounted_total

In [15]:
display(HTML('<table><tr><th>Network proportion taken into consideration</th><th>Coal</th><th>Gas</th><th>Oil</th><th>Hydro</th><th>Renewable</th><th>Nuclear</th></tr>'
        '<tr><td>{:.02f}%</td><td>'.format(accounted_total*100) +\
        '</td><td>'.join('{:.02f}%'.format(x*100) for x in fracs) + '</td></tr></table>'))

Network proportion taken into consideration,Coal,Gas,Oil,Hydro,Renewable,Nuclear
67.35%,34.17%,21.25%,0.92%,9.02%,10.49%,23.40%


In [16]:
coal_kg_per_kwh = 1.04 * 0.45359237 # kg
gas_l_per_kwh = 0.01011 * 28.316846592 # l
oil_l_per_kwh = 0.00173 * 119.240471196 # l
uranium_g_per_kwh = 1/50 /24 *1e-3

* https://www.eia.gov/tools/faqs/faq.cfm?id=667&t=3
* https://en.wikipedia.org/wiki/United_States_customary_units#Fluid_volume
* https://en.wikipedia.org/wiki/Cubic_foot
* https://en.wikipedia.org/wiki/Pound_(mass)
* http://www.eea.europa.eu/data-and-maps/indicators/nuclear-energy-and-waste-production/nuclear-energy-and-waste-production-3

In [17]:
# g/GJ: CO2, SO2, NOX, CO, particulate matter
pollutants_coal = np.array([94600, 765, 292, 89.1, 1203])
pollutants_oil = np.array([77400, 1350, 195, 15.7, 16])
pollutants_gas = np.array([56100, 0.68, 93.3, 14.5, 0.1])

* https://en.wikipedia.org/wiki/Fossil-fuel_power_station#Environmental_impacts

In [18]:
def res_table_template(units, wh_prefix, wh_vals, x_values):
    u_coal, u_gas, u_oil, u_nuclear = units
    wh_coal, wh_gas, wh_oil, wh_hydro, wh_renewable, wh_nuclear = wh_vals
    x_coal, x_gas, x_oil, x_nuclear = x_values
    wh = wh_prefix+'Wh'
    display(HTML('''
<table><tr><th colspan=2>Coal</th><th colspan=2>Gas</th><th colspan=2>Oil</th><th colspan=2>Nuclear</th><th>Hydro</th><th>Renewable</th></tr>
<tr><th>{wh}</th><th>{u_coal}</th><th>{wh}</th><th>{u_gas}</th><th>{wh}</th><th>{u_oil}</th><th>{wh}</th><th>{u_nuclear} Uranium</th><th>{wh}</th><th>{wh}</th></tr>
<tr><td>{wh_coal:.02f}</td><td>{x_coal:.02f}</td>
<td>{wh_gas:.02f}</td><td>{x_gas:.02f}</td>
<td>{wh_oil:.02f}</td><td>{x_oil:.02f}</td>
<td>{wh_nuclear:.02f}</td><td>{x_nuclear:.02f}</td>
<td>{wh_renewable:.02f}</td>
<td>{wh_hydro:.02f}</td>
</tr></table>
'''.format(**locals())))
    
def pollutant_table_template(units, amounts):
    u_co2, u_so2, u_nox, u_co, u_part = units
    x_co2, x_so2, x_nox, x_co, x_part = amounts
    display(HTML('''
<table><tr><th>CO_2</th><th>SO_2</th><th>NO_x</th><th>CO</th><th>Particulate matter</th></tr>
<tr><td>{x_co2:.02f}{u_co2}</td>
<td>{x_so2:.02f}{u_so2}</td>
<td>{x_nox:.02f}{u_nox}</td>
<td>{x_co:.02f}{u_co}</td>
<td>{x_part:.02f}{u_part}</td>
</tr></table>
'''.format(**locals())))

total_pollutants = (pollutants_coal*fracs[0] + pollutants_gas*fracs[1] + pollutants_oil*fracs[2])*1e-3 # kg/GJ

pollutants_per_tx = energy_per_tx*1e-9 * total_pollutants # GJ * kg/GJ → kg
display(HTML('<h3>Values per transaction</h3>'))
res_table_template(['kg', 'l', 'ml', 'ug'], 'k',
        fracs * kwh_per_tx,
        [coal_kg_per_kwh * kwh_per_tx * fracs[0],
         gas_l_per_kwh * kwh_per_tx * fracs[1],
         oil_l_per_kwh * kwh_per_tx * fracs[2] * 1e3,
         uranium_g_per_kwh * kwh_per_tx * fracs[5] * 1e6 # ug
        ])
pollutant_table_template(['kg', 'g', 'g', 'g', 'g'], np.multiply(pollutants_per_tx, np.array([1, 1e3, 1e3, 1e3, 1e3])))

network_kw = network_power_consumption/1e3 # kW
pollutants_per_h = network_power_consumption*1e-9*3600 * total_pollutants # GW*s * kg/GJ → kg
display(HTML('<h3>Values per hour</h3>'))
res_table_template(['t', 'm^3', 'hl', 'mg'], 'M',
        fracs * network_kw * 1e-3, # MWh
        [coal_kg_per_kwh * network_kw * fracs[0] * 1e-3, # t
         gas_l_per_kwh * network_kw * fracs[1] * 1e-3, # m^3
         oil_l_per_kwh * network_kw * fracs[2] * 1e-2, # hl
         uranium_g_per_kwh * network_kw * fracs[5] * 1e3 # mg
        ])
pollutant_table_template(['t', 'kg', 'kg', 'kg', 'kg'], np.multiply(pollutants_per_h, np.array([1e-3, 1, 1, 1, 1])))

hr_per_yr = 365.25 * 24
display(HTML('<h3>Values per year</h3>'))
res_table_template(['Mt', '10^3 m^3', '10^6 l', 'g'], 'G',
        fracs *  network_kw * hr_per_yr* 1e-6, # GWh
        [coal_kg_per_kwh * hr_per_yr * network_kw * fracs[0] * 1e-6, # kt
         gas_l_per_kwh * hr_per_yr * network_kw * fracs[1] * 1e-6, # 10^6 m^3
         oil_l_per_kwh * hr_per_yr * network_kw * fracs[2] * 1e-6, # 10^9 l
         uranium_g_per_kwh * hr_per_yr * network_kw * fracs[5]
        ])
pollutant_table_template(['kt', 'kt', 't', 't', 'kt'], np.multiply(pollutants_per_h * hr_per_yr, np.array([1e-6, 1e-6, 1e-3, 1e-3, 1e-6])))

CO_2,SO_2,NO_x,CO,Particulate matter
2.83kg,17.21g,7.63g,2.12g,25.84g


CO_2,SO_2,NO_x,CO,Particulate matter
28.72t,175.00kg,77.55kg,21.51kg,262.72kg


CO_2,SO_2,NO_x,CO,Particulate matter
251.76kt,1.53kt,679.82t,188.57t,2.30kt


In [23]:
network_kw * hr_per_yr* 1e-6 # GWh per Year

1555.6365022173418

In [19]:
tx_per_d = avg_tx_last_1000 * 3600 * 24
tx_per_d

243971.5124249368

In [20]:
sum_inputs, sum_outputs, tx_count = 0, 0, 0
history_size = 1000
with db as conn:
    last_idx = db.execute('SELECT MAX(idx) FROM blockinfo').fetchone()[0]
    for b in blocks(last_idx-history_size, last_idx, conn):
        for tx in b['tx']:
            sum_inputs += len(tx['inputs'])
            sum_outputs += len(tx['out'])
            tx_count += 1
# Note: We don't need more precise statistical modelling here as we only want to do a load simulation, and for that the average value is all that matters.
avg_inputs = sum_inputs / tx_count
avg_outputs = sum_outputs / tx_count
avg_inputs, avg_outputs

(2.3942313410284384, 2.3947325625282923)

In [21]:
import sqlite3
class TransactionSimulation:
    def __init__(self, filename='/tmp/btc_tx_sim.db'):
        self.simdb = sqlite3.connect(filename)
        self.simdb.isolation_level = 'EXCLUSIVE'

    def populate_db(self, n_accounts, max_acct_val=1e12):
        chunk_size = int(min(100, n_acc))
        with self.simdb as conn:
            conn.execute('DROP TABLE IF EXISTS accounts')
            conn.execute('DROP TABLE IF EXISTS tx_meta')
            conn.execute('DROP TABLE IF EXISTS tx_xfer')
            conn.execute('CREATE TABLE accounts (metadata BLOB, balance INTEGER '
                         'CHECK(balance >= 0))')
            conn.execute('CREATE TABLE tx_meta (metadata BLOB)')
            conn.execute('CREATE TABLE tx_xfer (amount INTEGER, acct_id INTEGER, tx_meta_id INTEGER,'
                         'FOREIGN KEY(tx_meta_id) REFERENCES tx_meta(rowid),'
                         'FOREIGN KEY (acct_id) REFERENCES accounts(rowid))')
            conn.executemany('INSERT INTO accounts VALUES (?, ?)',
                    [ (np.random.bytes(64), np.random.randint(max_acct_val)) for _ in range(n_accounts) ])
    
    def execute_tx(self, inputs, outputs):
        with self.simdb as conn:
            try:
                cur = conn.cursor()
                cur.execute('INSERT INTO tx_meta VALUES (?)', (np.random.bytes(64),))
                tx_id = cur.lastrowid
                balance = 0

                for val, acct_id in inputs:
                    cur.execute('UPDATE accounts SET balance = balance - ? WHERE rowid=?', (val, acct_id))
                    cur.execute('INSERT INTO tx_xfer VALUES (?, ?, ?)', (val, acct_id, tx_id))
                    balance += val

                for val, acct_id in outputs:
                    cur.execute('UPDATE accounts SET balance = balance + ? WHERE rowid=?', (val, acct_id))
                    cur.execute('INSERT INTO tx_xfer VALUES (?, ?, ?)', (-val, acct_id, tx_id))
                    balance -= val

                if balance != 0:
                    raise ValueError('tx does not tally up')
            except:
                conn.rollback()
                raise

    def sim_tx(self, size_sim, avg_inputs=2.4, avg_outputs=2.4, max_tx_item=1e10, zipf_a=1.05):
        poisson_in = np.random.poisson(avg_inputs-1, size_sim)
        poisson_out = np.random.poisson(avg_outputs-1, size_sim)
        for n_in, n_out in zip(poisson_in, poisson_out):
            n_in, n_out = n_in + 1, n_out + 1
            
            inputs = np.random.zipf(zipf_a, n_in)
            outputs = np.random.zipf(zipf_a, n_out)
            vals_in = np.random.randint(1, max_tx_item, n_in)
            volume = sum(vals_in)
            # This is slightly skewed, but we don't care as this is only a toy anyway.
            dist_out = np.random.rand(n_out+1)
            np.sort(dist_out)
            dist_out[0], dist_out[-1] = 0.0, 1.0
            dist_out *= volume
            dist_out = dist_out.astype(np.int)
            vals_out = dist_out[1:] - dist_out[:-1]
            
            try:
                self.execute_tx(zip(vals_in, inputs), zip(vals_out, outputs))
            except:
                continue
                # for debugging
                print(inputs)
                print(vals_in)
                print(outputs)
                print(vals_out)
                print(sum(vals_in), sum(vals_out))
                raise

sim = TransactionSimulation()
size_sim = int(tx_per_d)

print('Populating test database...', flush=True)
#sim.populate_db(size_sim*5)
print('done.', flush=True)

print('Running test...', flush=True)
time_start = time.time()
sim.sim_tx(size_sim, avg_inputs, avg_outputs)
time_end = time.time()
test_tx_per_s = size_sim/(time_end-time_start)
print('done. {:.02f} tx/s'.format(test_tx_per_s))

Populating test database...
done.
Running test...
done. 4004.90 tx/s


In [22]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

speed_factor = (test_tx_per_s / avg_tx_last_1000)
power_factor = (65 / network_power_consumption)
efficiency_factor = speed_factor / power_factor

print('speedup:', locale.format('%d', speed_factor, grouping=True))
print('power consumption: {:.02f}ppm'.format(power_factor*1e6))
print('efficiency:', locale.format('%d', efficiency_factor, grouping=True)+'x')

speedup: 1,418
power consumption: 0.37ppm
efficiency: 3,872,216,511x
